In [1]:
from commom.database.queries.query_lojas import QUERY_LOJAS
from commom.database.queries.query_vendas_nf_novo import QUERY_VENDAS_NF_NOVO
from commom.kpi_data.kpi_data_manager import DataHandler, KpiDataManager

manager = KpiDataManager()
handler = DataHandler()


In [2]:
df = handler.read_from_bigquery(QUERY_VENDAS_NF_NOVO)
df

Forbidden: 403 Access Denied: Table pix-engenhariadados:ammo_varejo_pdv.delivery_from_distributor: User does not have permission to query table pix-engenhariadados:ammo_varejo_pdv.delivery_from_distributor, or perhaps it does not exist.; reason: accessDenied, message: Access Denied: Table pix-engenhariadados:ammo_varejo_pdv.delivery_from_distributor: User does not have permission to query table pix-engenhariadados:ammo_varejo_pdv.delivery_from_distributor, or perhaps it does not exist.

Location: US
Job ID: 42b2e50b-603c-4b3e-8b25-61bf7ad210f1
